<a href="https://colab.research.google.com/github/monoramasn/Speech_fairness/blob/main/cs_whisper_small_test_(edit).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install git+https://github.com/huggingface/transformers
!pip install librosa
!pip install evaluate>=0.30
!pip install jiwer
!pip install gradio

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-o1xirwz8
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-o1xirwz8
  Resolved https://github.com/huggingface/transformers to commit ec43d6870aa1afb42a6d2b1b0a03743d3f9b3ce6
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.36.0.dev0-py3-none-any.whl size=8234907 sha256=18324324e5754c2f79ccda23639c8796eb4d6af59c13bb9653ecdff9e43985f2
  Stored in directory: /tmp/pip-ephem-wheel-cache-e6bnjx9w/wheels/c0/14/d6/6c9a5582d2ac191ec0a483be151a4495fe1eb2a6706ca49f1b
Successfully built transformers
  Attempting uninstall: transformers
    Found existing installation: transformers 4.35.2
    Uninstalling transformers-4.35.2:
      Successfully uninstalled transformers-4.35.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import torch
from transformers import AutoFeatureExtractor, WhisperModel
from datasets import load_dataset

In [3]:
import os
import numpy as np

try:
    import tensorflow  # required in Colab to avoid protobuf compatibility issues
except ImportError:
    pass

import torch
import pandas as pd
import torchaudio

from tqdm.notebook import tqdm

In [4]:
voxpopuli_data = load_dataset("facebook/voxpopuli", "cs", split="test")
#column_name = "audio, normalized_text"

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [5]:
voxpopuli_data

Dataset({
    features: ['audio_id', 'language', 'audio', 'raw_text', 'normalized_text', 'gender', 'speaker_id', 'is_gold_transcript', 'accent'],
    num_rows: 1123
})

In [6]:
!pip install openai-whisper
import whisper
#model = whisper.load_model("base")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 798.6/798.6 kB 14.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 49.9 MB/s eta 0:00:00
  Created wheel for openai-whisper: filename=openai_whisper-20231117-py3-none-any.whl size=801356 sha256=7a30b17d0dedfd60b239c023e09339f73c682391a4221bd2c67f152104175626
  Stored in directory: /root/.cache/pip/wheels/d0/85/e1/9361b4cbea7dd4b7f6702fa4c3afc94877952eeb2b62f45f56
Successfully built openai-whisper
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires openai, which is not installed.


In [33]:
model = whisper.load_model("small")

100%|███████████████████████████████████████| 461M/461M [00:07<00:00, 67.8MiB/s]


In [35]:
from transformers import WhisperForConditionalGeneration, WhisperFeatureExtractor,WhisperTokenizer
from datasets import load_dataset
import torch
from tqdm import tqdm
from jiwer import wer

In [36]:
Gender=voxpopuli_data['gender']
Audio=voxpopuli_data['audio']
text=voxpopuli_data['normalized_text']
Gender=pd.DataFrame(Gender)
Audio=pd.DataFrame(Audio)
text=pd.DataFrame(text)

In [37]:
text

,0
0,tato norma posiluje pozici evropské agentury p...
1,dámy a pánové útok na židy je útok na nás samo...
2,zatímco se na půdě evropských institucí dohodl...
3,paní předsedající když hovoříme o humanitární ...
4,vážené poslankyně vážení poslanci zastupuji dn...
...,...
1118,ale vzhledem ke stárnutí populace a dalším výz...
1119,pane předsedající já myslím že můj přítel rich...
1120,erasmus.
1121,do budoucna je zřejmé a varují před tím analýz...


In [38]:
Gender.columns=['Gender']
text.columns=['Transcript']

In [39]:
file=pd.concat([Gender, text, Audio],axis=1)
file

,Gender,Transcript,path,array,sampling_rate
0,male,tato norma posiluje pozici evropské agentury p...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.000274658203125, -0.00067138671875, -0.001...",16000
1,male,dámy a pánové útok na židy je útok na nás samo...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.001861572265625, 0.000213623046875, 0.0020...",16000
2,male,zatímco se na půdě evropských institucí dohodl...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.001495361328125, -0.00152587890625, -0.0089...",16000
3,male,paní předsedající když hovoříme o humanitární ...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.00018310546875, -6.103515625e-05, 0.000762...",16000
4,male,vážené poslankyně vážení poslanci zastupuji dn...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.0001220703125, -0.000152587890625, -0.00015...",16000
...,...,...,...,...,...
1118,female,ale vzhledem ke stárnutí populace a dalším výz...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.00335693359375, -0.01141357421875, 0.00012...",16000
1119,male,pane předsedající já myslím že můj přítel rich...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.00079345703125, -0.001068115234375, -0.001...",16000
1120,male,erasmus.,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.003448486328125, -0.00347900390625, -0.004...",16000
1121,male,do budoucna je zřejmé a varují před tím analýz...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.000335693359375, 0.00469970703125, 0.00384...",16000


In [40]:
male_samples=file.loc[file['Gender'] == 'male']
male_samples

,Gender,Transcript,path,array,sampling_rate
0,male,tato norma posiluje pozici evropské agentury p...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.000274658203125, -0.00067138671875, -0.001...",16000
1,male,dámy a pánové útok na židy je útok na nás samo...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.001861572265625, 0.000213623046875, 0.0020...",16000
2,male,zatímco se na půdě evropských institucí dohodl...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.001495361328125, -0.00152587890625, -0.0089...",16000
3,male,paní předsedající když hovoříme o humanitární ...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.00018310546875, -6.103515625e-05, 0.000762...",16000
4,male,vážené poslankyně vážení poslanci zastupuji dn...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.0001220703125, -0.000152587890625, -0.00015...",16000
...,...,...,...,...,...
1116,male,já když jsem slyšel pana komisaře jak si vymez...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.001617431640625, -0.00146484375, 6.1035156...",16000
1119,male,pane předsedající já myslím že můj přítel rich...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.00079345703125, -0.001068115234375, -0.001...",16000
1120,male,erasmus.,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.003448486328125, -0.00347900390625, -0.004...",16000
1121,male,do budoucna je zřejmé a varují před tím analýz...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.000335693359375, 0.00469970703125, 0.00384...",16000


In [41]:
female_samples=file.loc[file['Gender'] == 'female']
female_samples

,Gender,Transcript,path,array,sampling_rate
8,female,nejhorší situace je v somálsku súdánu mauretán...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.001007080078125, -0.001922607421875, -0.00...",16000
9,female,v letecké i železniční dopravě vize nula platí...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.001800537109375, 0.0030517578125, 0.0003356...",16000
10,female,jen tak potvrdíte že před dvaceti lety lidé ne...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.001007080078125, 0.0009765625, -0.00057983...",16000
11,female,v této souvislosti bych očekávala že alespoň d...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.000518798828125, -0.0020751953125, -0.0010...",16000
13,female,já sama pocházím z regionu severní moravy ze s...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.000457763671875, 0.00201416015625, 0.00283...",16000
...,...,...,...,...,...
1106,female,díky tomu se mnoho informací dostává k adresát...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.000244140625, -9.1552734375e-05, 0.0009155...",16000
1109,female,podle dostupných informací turecký premiér dal...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.00506591796875, -0.008453369140625, -0.003...",16000
1114,female,veřejné konzultace jsou běžně zveřejňovány pře...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.001251220703125, 0.00177001953125, -0.00122...",16000
1117,female,liší se rozsah průvodních dokumentů opět vesmě...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.00396728515625, 0.003875732421875, 0.005584...",16000


In [42]:
def transcribe_audio(file_path):
    result = model.transcribe(file_path, language="cs", fp16=False)
    return result["text"]

In [43]:
def calculate_wer(ground_truths, hypotheses):
    return wer(ground_truths, hypotheses)

In [44]:
from jiwer import wer
from collections import Counter

def calculate_precision_recall_f1(ground_truth, hypothesis):
    ground_truth_words = set(ground_truth.split())
    hypothesis_words = set(hypothesis.split())

    true_positives = ground_truth_words.intersection(hypothesis_words)
    false_positives = hypothesis_words - ground_truth_words
    false_negatives = ground_truth_words - hypothesis_words

    precision = len(true_positives) / (len(true_positives) + len(false_positives)) if true_positives or false_positives else 0
    recall = len(true_positives) / (len(true_positives) + len(false_negatives)) if true_positives or false_negatives else 0
    f1_score = 2 * (precision * recall) / (precision + recall) if precision + recall else 0

    return precision, recall, f1_score

In [45]:

total_wer = 0
total_precision = 0
total_recall = 0
total_f1 = 0
num_sample = 0

for index,row in male_samples.iterrows():
    audio_path=row['path']
    ground_truth_text = row['Transcript']
    transcription = transcribe_audio(audio_path)
    # Compute metrics
    sample_wer = calculate_wer(ground_truth_text, transcription)
    precision, recall, f1_score = calculate_precision_recall_f1(ground_truth_text, transcription)

    total_wer += sample_wer
    total_precision += precision
    total_recall += recall
    total_f1 += f1_score
    num_sample += 1

    print(f"Processed {num_sample} files, WER: {sample_wer}, Precision: {precision}, Recall: {recall}, F1: {f1_score}")

# Calculate average metrics
average_wer = total_wer / num_sample
average_precision = total_precision / num_sample
average_recall = total_recall / num_sample
average_f1 = total_f1 / num_sample

print(f"Average WER: {average_wer}")
print(f"Average Precision: {average_precision}")
print(f"Average Recall: {average_recall}")
print(f"Average F1-Score: {average_f1}")

Processed 1 files, WER: 0.25806451612903225, Precision: 0.7777777777777778, Recall: 0.7241379310344828, F1: 0.75
Processed 2 files, WER: 0.8181818181818182, Precision: 0.2857142857142857, Recall: 0.2222222222222222, F1: 0.25
Processed 3 files, WER: 0.6774193548387096, Precision: 0.4, Recall: 0.41379310344827586, F1: 0.4067796610169491
Processed 4 files, WER: 0.38636363636363635, Precision: 0.625, Recall: 0.6944444444444444, F1: 0.6578947368421053
Processed 5 files, WER: 1.0, Precision: 0.0, Recall: 0.0, F1: 0
Processed 6 files, WER: 0.5151515151515151, Precision: 0.5555555555555556, Recall: 0.46875, F1: 0.5084745762711864
Processed 7 files, WER: 0.46153846153846156, Precision: 0.5833333333333334, Recall: 0.5833333333333334, F1: 0.5833333333333334
Processed 8 files, WER: 0.5454545454545454, Precision: 0.5, Recall: 0.45454545454545453, F1: 0.47619047619047616
Processed 9 files, WER: 0.44680851063829785, Precision: 0.7027027027027027, Recall: 0.6666666666666666, F1: 0.6842105263157895
Pro

In [46]:
total_wer = 0
total_precision = 0
total_recall = 0
total_f1 = 0
num_sample = 0

for index,row in female_samples.iterrows():
    audio_path=row['path']
    ground_truth_text = row['Transcript']
    transcription = transcribe_audio(audio_path)
    # Compute metrics
    sample_wer = calculate_wer(ground_truth_text, transcription)
    precision, recall, f1_score = calculate_precision_recall_f1(ground_truth_text, transcription)


    total_wer += sample_wer
    total_precision += precision
    total_recall += recall
    total_f1 += f1_score
    num_sample += 1

    print(f"Processed {num_sample} files, WER: {sample_wer}, Precision: {precision}, Recall: {recall}, F1: {f1_score}")

# Calculate average metrics
average_wer = total_wer / num_sample
average_precision = total_precision / num_sample
average_recall = total_recall / num_sample
average_f1 = total_f1 / num_sample

print(f"Average WER: {average_wer}")
print(f"Average Precision: {average_precision}")
print(f"Average Recall: {average_recall}")
print(f"Average F1-Score: {average_f1}")

Processed 1 files, WER: 0.3333333333333333, Precision: 0.6666666666666666, Recall: 0.6666666666666666, F1: 0.6666666666666666
Processed 2 files, WER: 0.5, Precision: 0.5294117647058824, Recall: 0.5294117647058824, F1: 0.5294117647058824
Processed 3 files, WER: 0.23076923076923078, Precision: 0.7692307692307693, Recall: 0.7692307692307693, F1: 0.7692307692307693
Processed 4 files, WER: 0.35294117647058826, Precision: 0.6571428571428571, Recall: 0.696969696969697, F1: 0.676470588235294
Processed 5 files, WER: 0.55, Precision: 0.47368421052631576, Recall: 0.47368421052631576, F1: 0.47368421052631576
Processed 6 files, WER: 0.2777777777777778, Precision: 0.7058823529411765, Recall: 0.7058823529411765, F1: 0.7058823529411765
Processed 7 files, WER: 0.6666666666666666, Precision: 0.4117647058823529, Recall: 0.4666666666666667, F1: 0.43749999999999994
Processed 8 files, WER: 0.14285714285714285, Precision: 0.8571428571428571, Recall: 0.8571428571428571, F1: 0.8571428571428571
Processed 9 file